In [1]:
import numpy as np

In [2]:
def forwardMultiplyGate(x, y):
    return x * y

x = -2
y = 3
print(forwardMultiplyGate(x, y))

-6


### The Goal
The problem we are interested in studying looks as follows:

We provide a given circuit some specific input values (e.g. x = -2, y = 3)
The circuit computes an output value (e.g. -6)
The core question then becomes: How should one tweak the input slightly to increase the output?
In this case, in what direction should we change x,y to get a number larger than -6? Note that, for example, x = -1.99 and y = 2.99 gives x * y = -5.95, which is higher than -6.0. Don’t get confused by this: -5.95 is better (higher) than -6.0. It’s an improvement of 0.05, even though the magnitude of -5.95 (the distance from zero) happens to be lower.

## Strategy #1: Random Local Search

In [3]:
tweakAmount = 0.01
best = -np.inf
bestX = x
bestY = y

for k in range(0, 100):
    x_try = x + tweakAmount * (np.random.uniform() * 2 - 1)
    y_try = y + tweakAmount * (np.random.uniform() * 2 - 1)
    out = forwardMultiplyGate(x_try, y_try)
    if out > best:
        best = out
        bestX = x_try
        bestY = y_try

In [4]:
print(f"Best x: {bestX}")
print(f"Best y: {bestY}")
print(f"Best out: {best}")

Best x: -1.990797610736487
Best y: 2.9907113917947616
Best out: -5.9539010931874055


#### This is a perfectly fine strategy for tiny problems with a few gates if you can afford the compute time, but it won’t do if we want to eventually consider huge circuits with millions of inputs. It turns out that we can do much better.

## Strategy #2: Numerical Gradient


In [5]:
x = -2
y = 3
out = forwardMultiplyGate(x, y)
h = 0.0001

# Compute derivative with respect to x
xph = x + h
out2 = forwardMultiplyGate(xph, y)
x_derivative = (out2 - out) / h

# compute derivative with respect to y
yph = y + h
out3 = forwardMultiplyGate(x, yph)
y_derivative = (out3 - out) / h

print(f"X_deriv: {x_derivative}")
print(f"Y_deriv: {y_derivative}")

print(f"X_deriv out: {out2}")
print(f"Y_deriv out: {out3}")

X_deriv: 3.00000000000189
Y_deriv: -2.0000000000042206
X_deriv out: -5.9997
Y_deriv out: -6.0002


In [6]:
stepSize = 0.01
out = forwardMultiplyGate(x, y)  # -6
x = x + stepSize * x_derivative
y = y + stepSize * y_derivative
outNew = forwardMultiplyGate(x, y)

print(x)
print(y)
print(outNew)

-1.969999999999981
2.979999999999958
-5.87059999999986


## Strategy #3: Analytic Gradient

In [7]:
x = -2
y = 3
out = forwardMultiplyGate(x, y)
x_gradient = y
y_gradient = x

stepSize = 0.01
x += stepSize * x_gradient
y += stepSize * y_gradient
outNew = forwardMultiplyGate(x, y)

print(x)
print(y)
print(outNew)

-1.97
2.98
-5.8706


## Recursive Case: Circuits with Multiple Gates

The expression we are computing now is f(x,y,z)=(x+y)z.

In [8]:
def forwardMultiplyGate(a, b):
    return a * b

def forwardAddGate(a, b):
    return a + b

def forwardCircuit(x, y, z):
    q = forwardAddGate(x, y)
    f = forwardMultiplyGate(q, z)
    return f

x = -2
y = 5
z = -4
f = forwardCircuit(x, y, z)
print(f)

-12


### Backpropagation

In [9]:
# Initial conditions
x = -2
y = 5
z = -4
q = forwardAddGate(x, y)
f = forwardMultiplyGate(q, z)

# gradient of the MULTIPLY gate with respect to its inputs
derivative_f_wrt_z = q  # 3
derivative_f_wrt_q = z  # -4

# derivative of the ADD gate with respect to its inputs
derivative_q_wrt_x = 1.0
derivative_q_wrt_y = 1.0

# chain rule
derivative_f_wrt_x = derivative_q_wrt_x * derivative_f_wrt_q  # -4
derivative_f_wrt_y = derivative_q_wrt_y * derivative_f_wrt_q  # -4
print(derivative_f_wrt_x)
print(derivative_f_wrt_y)

-4.0
-4.0


In [10]:
# final gradient from above: [-4, -4, 3]
gradient_f_wrt_xyz = [derivative_f_wrt_x, derivative_f_wrt_y, derivative_f_wrt_z]

# leth the inputs respond to the force/tug:
stepSize = 0.01
x = x + stepSize * derivative_f_wrt_x
y = y + stepSize * derivative_f_wrt_y
z = z + stepSize * derivative_f_wrt_z
print(f"new x: {x}")
print(f"new y: {y}")
print(f"new z: {z}")

# Our circuit noew better give higher output:
q = forwardAddGate(x, y)
f = forwardMultiplyGate(q, z)
print(f"new forwardAddGate out: {q}")
print(f"new forwardMultiplyGate out: {f}")

new x: -2.04
new y: 4.96
new z: -3.97
new forwardAddGate out: 2.92
new forwardMultiplyGate out: -11.5924


In [11]:
# Numerical gradient check
x = -2
y = 5
z = -4

# Numerical gradient check
h = 0.0001
x_derivative = (forwardCircuit(x + h, y, z) - forwardCircuit(x, y, z)) / h
y_derivative = (forwardCircuit(x, y + h, z) - forwardCircuit(x, y, z)) / h
z_derivative = (forwardCircuit(x, y, z + h) - forwardCircuit(x, y, z)) / h

print([x_derivative, y_derivative, z_derivative])

[-3.9999999999906777, -3.9999999999906777, 3.000000000010772]


# Example Neuron

In [12]:
# a Unit corresponding to a wire in diagrams between gates
class Unit:
    def __init__(self, value, grad):
        self.value = value
        self.grad = grad


class MultiplyGate:
    def forward(self, u0, u1):
        self.u0 = u0
        self.u1 = u1
        self.utop = Unit(u0.value * u1.value, 0.0)
        return self.utop

    def backward(self):
        self.u0.grad += self.u1.value * self.utop.grad
        self.u1.grad += self.u0.value * self.utop.grad


class AddGate:
    def forward(self, u0, u1):
        self.u0 = u0
        self.u1 = u1
        self.utop = Unit(u0.value + u1.value, 0.0)
        return self.utop

    def backward(self):
        self.u0.grad += 1 * self.utop.grad
        self.u1.grad += 1 * self.utop.grad


class SigmoidGate:
    def sig(self, x):
        return 1 / (1 + np.exp(-x))

    def forward(self, u0):
        self.u0 = u0
        self.utop = Unit(self.sig(self.u0.value), 0.0)
        return self.utop

    def backward(self):
        s = self.sig(self.u0.value)
        self.u0.grad += (s * (1 - s)) * self.utop.grad

In [13]:
# Create input units
a = Unit(1.0, 0.0)
b = Unit(2.0, 0.0)
c = Unit(-3.0, 0.0)
x = Unit(-1.0, 0.0)
y = Unit(3.0, 0.0)

# Create the gates
mulg0 = MultiplyGate()
mulg1 = MultiplyGate()
addg0 = AddGate()
addg1 = AddGate()
sg0 = SigmoidGate()

# Forward pass
def forwardNeuron(a, b, c, x, y):
    ax = mulg0.forward(a, x)
    by = mulg1.forward(b, y)
    ax_by = addg0.forward(ax, by)
    ax_by_c = addg1.forward(ax_by, c)
    s = sg0.forward(ax_by_c)
    return s

s = forwardNeuron(a, b, c, x, y)
print(s.value)

0.8807970779778823


In [14]:
s.grad = 1.0
sg0.backward()    # writes gradient into ax_by_c
addg1.backward()  # writes gradients into ax_by and c
addg0.backward()  # writes gradients into ax and by
mulg1.backward()  # writes gradients into b and y
mulg0.backward()  # writes gradients into a and x

In [15]:
print(f"a.grad is: {a.grad}")
print(f"b.grad is: {b.grad}")
print(f"c.grad is: {c.grad}")
print(f"x.grad is: {x.grad}")
print(f"y.grad is: {y.grad}")

stepSize = 0.01
a.value += stepSize * a.grad 
b.value += stepSize * b.grad  
c.value += stepSize * c.grad  
x.value += stepSize * x.grad 
y.value += stepSize * y.grad 

s = forwardNeuron(a, b, c, x, y)
print(f"Circuit output after one backprop: {s.value}")

a.grad is: -0.10499358540350662
b.grad is: 0.31498075621051985
c.grad is: 0.10499358540350662
x.grad is: 0.10499358540350662
y.grad is: 0.20998717080701323
Circuit output after one backprop: 0.8825501816218984


In [16]:
def forwardCircuitFast(a, b, c, x, y):
    return 1 / (1 + np.exp(-(a*x + b*y + c)))

a = 1
b = 2
c = -3
x = -1
y = 3
h = 0.0001
a_grad = (forwardCircuitFast(a+h, b, c, x, y) - forwardCircuitFast(a, b, c, x, y)) / h
b_grad = (forwardCircuitFast(a, b+h, c, x, y) - forwardCircuitFast(a, b, c, x, y)) / h
c_grad = (forwardCircuitFast(a, b, c+h, x, y) - forwardCircuitFast(a, b, c, x, y)) / h
x_grad = (forwardCircuitFast(a, b, c, x+h, y) - forwardCircuitFast(a, b, c, x, y)) / h
y_grad = (forwardCircuitFast(a, b, c, x, y+h) - forwardCircuitFast(a, b, c, x, y)) / h

print([a_grad, b_grad, c_grad, x_grad, y_grad])

[-0.10499758359205913, 0.3149447748351797, 0.10498958734506125, 0.10498958734506125, 0.2099711788272618]


# Machine Learning
___
## Binary Classification
### Support Vector Machine

In [28]:
# A circuit: it takes 5 Units (x, y, a, b, c) and outputs a single Unit
# It can also compute the gradient w.r.t its inputs
class Circuit:
    def __init__(self):
        # Create some gates
        self.mulg0 = MultiplyGate()
        self.mulg1 = MultiplyGate()
        self.addg0 = AddGate()
        self.addg1 = AddGate()
    
    def forward(self, x, y, a, b, c):
        self.ax = self.mulg0.forward(a, x)
        self.by = self.mulg1.forward(b, y)
        self.ax_by = self.addg0.forward(self.ax, self.by)
        self.ax_by_c = self.addg1.forward(self.ax_by, c)
        return self.ax_by_c
    
    def backward(self, gradientTop):  # Takes pull from above
        self.ax_by_c.grad = gradientTop
        self.addg1.backward()  # sets gradient in ax_by and c
        self.addg0.backward()  # sets gradient in ax and by
        self.mulg1.backward()  # sets gradient in b and y
        self.mulg0.backward()  # sets gradient in a and x

In [44]:
class SVM:
    def __init__(self):
        # Random initial parameter values
        self.a = Unit(1.0, 0.0)
        self.b = Unit(-2.0, 0.0)
        self.c = Unit(-1.0, 0.0)
        self.circuit = Circuit()
    
    def forward(self, x, y):
        self.unit_out = self.circuit.forward(x, y, self.a, self.b, self.c)
        return self.unit_out
    
    def backward(self, label):  # label is +1/-1
        # Reset pulls on a,b,c
        self.a.grad = 0.0
        self.b.grad = 0.0
        self.c.grad = 0.0

        # Compute the pull based on what the circuit output was
        pull = 0.0
        if label == 1 and self.unit_out.value < 1:
            pull = 1
        if label == -1 and self.unit_out.value > -1:
            pull = -1
        self.circuit.backward(pull)  # write gradient into x,y,a,b,c

        # Add regularization pull for parameters: towards zero and proportional to value
        self.a.grad += -self.a.value
        self.b.grad += -self.b.value
    
    def learnFrom(self, x, y, label):
        self.forward(x, y)  # forward pass (set .value in all Units)
        self.backward(label)  # backward pass (set .grad in all Units)
        self.parameterUpdate()  # parameters respond to tug
    
    def parameterUpdate(self):
        stepSize = 0.01
        self.a.value += stepSize * self.a.grad
        self.b.value += stepSize * self.b.grad
        self.c.value += stepSize * self.c.grad

In [53]:
data = []
labels = []
data.append([1.2, 0.7])
labels.append(1)
data.append([-0.3, -0.5])
labels.append(-1)
data.append([3.0, 0.1])
labels.append(1)
data.append([-0.1, -1.0])
labels.append(-1)
data.append([-1.0, 1.1])
labels.append(-1)
data.append([2.1, -3])
labels.append(1)

svm = SVM()

# a function that computes the classification accuracy
def evalTrainAcc():
    num_correct = 0
    for i in range(0, len(data)):
        x = Unit(data[i][0], 0.0)
        y = Unit(data[i][1], 0.0)
        trueLabel = labels[i]

        # see if the prediction matches the provided label
        predictedLabel = svm.forward(x, y).value > 0 if 1 else -1
        if predictedLabel == trueLabel:
            num_correct += 1
    return num_correct / len(data)

import math

# Training loop
for k in range(0, 1000):
    # Pick a random data point
    i = math.floor(np.random.uniform() * len(data))
    x = Unit(data[i][0], 0.0)
    y = Unit(data[i][1], 0.0)
    label = labels[i]
    svm.learnFrom(x, y, label)

    if k % 25 == 0:
        print(f"Training accuracy at iteration {k}: {evalTrainAcc()}")


Training accuracy at iteration 0: 0.3333333333333333
Training accuracy at iteration 25: 0.3333333333333333
Training accuracy at iteration 50: 0.3333333333333333
Training accuracy at iteration 75: 0.3333333333333333
Training accuracy at iteration 100: 0.3333333333333333
Training accuracy at iteration 125: 0.3333333333333333
Training accuracy at iteration 150: 0.3333333333333333
Training accuracy at iteration 175: 0.3333333333333333
Training accuracy at iteration 200: 0.3333333333333333
Training accuracy at iteration 225: 0.3333333333333333
Training accuracy at iteration 250: 0.3333333333333333
Training accuracy at iteration 275: 0.3333333333333333
Training accuracy at iteration 300: 0.3333333333333333
Training accuracy at iteration 325: 0.3333333333333333
Training accuracy at iteration 350: 0.3333333333333333
Training accuracy at iteration 375: 0.3333333333333333
Training accuracy at iteration 400: 0.3333333333333333
Training accuracy at iteration 425: 0.3333333333333333
Training accura

## Loss function

In [62]:
X = [ [1.2, 0.7], [-0.3, 0.5], [3, 2.5] ]
y = [1, -1, 1]
w = [0.1, 0.2, 0.3]
alpha = 0.1

def cost(X, y, w):
    totalCost = 0.0
    N = len(X)
    for i in range(0, N):
        # Loop over all data points and compute their score
        xi = X[i]
        score = w[0] * xi[0] + w[1] * xi[1] + w[2]

        # Accumulate cost based on how compatible the score is with the label
        yi = y[i]
        costi = np.max([0, -yi * score + 1])
        print(f"example {i}: xi = ({xi}) and label = {yi}")
        print(f"Score computed to be {score}")
        print(f"=> Cost computed to be {costi}")
        totalCost += costi

    # Regularization cost: we want small weights
    regCost = alpha * (w[0]*w[0] + w[1]*w[1])
    print(f"Regularization cost for current model is {regCost}")
    totalCost += regCost

    print(f"Total cost is {totalCost}")
    return totalCost

cost(X, y, w)

example 0: xi = ([1.2, 0.7]) and label = 1
Score computed to be 0.56
=> Cost computed to be 0.43999999999999995
example 1: xi = ([-0.3, 0.5]) and label = -1
Score computed to be 0.37
=> Cost computed to be 1.37
example 2: xi = ([3, 2.5]) and label = 1
Score computed to be 1.1
=> Cost computed to be 0.0
Regularization cost for current model is 0.005000000000000001
Total cost is 1.815


1.815